# Домашнее задание 1. Часть 2. Классификация.

В этом задании потребуется обучить классификатор изображений. Будем работать с датасетом, название которого раскрывать не будем. Можете посмотреть самостоятельно на картинки, которые в датасете есть. В нём 200 классов и около 5 тысяч картинок на каждый класс. Классы пронумерованы, как нетрудно догадаться, от 0 до 199. Скачать датасет можно вот [тут](https://yadi.sk/d/BNR41Vu3y0c7qA).

Структура датасета простая -- есть директории train и val, в которых лежат обучающие и валидационные данные. В train/ и val/ лежат директориии, соответствующие классам изображений, в которых лежат собственно сами изображения.
 
__Задание__. Добейтесь accuracy **на валидации не менее 0.44**. Напишите краткий отчёт о проделанных экспериментах. Что сработало и что не сработало? Почему вы решили, сделать так, а не иначе? Обязательно указывайте ссылки на чужой код, если вы его используете. Обязательно ссылайтесь на статьи/блогпосты/вопросы на stackoverflow/видосы от (индийских) ютуберов/курсы/подсказки от Дяди Васи и прочие дополнительные материалы, если вы их используете. 

В коде ниже необходимо, чтобы код проходил все `assert`'ы.

Необходимо написать функцию `predict` по шаблону ниже. Эта функция принимает на вход модель, даталоадер с валидационнами данными, criterion для подсчёта лосса и device, на котором будут производиться вычисления (определён ниже) и возвращает список лоссов по всем объектам, список из предсказанных классов для каждого объекта из из даталоалера и список из настоящих классов для каждого объекта в даталоадере (и именно в таком порядке).

__Использовать внешние данные для обучения строго запрещено__. Можно использовать предобученные модели из `torchvision`.

__Критерии оценки__: Оценка вычисляется по простой формуле: min(5, 5 * Ваша accuracy / 0.44). Оценка округляется до десятых по арифметическим правилам.

__Советы и указания__:
 - Наверняка вам потребуется много гуглить о классификации и о том, как заставить её работать. Это нормально, все гуглят. Но не забывайте, что нужно быть готовым за скатанный код отвечать на защите :)
 - Используйте аугментации. Для этого пользуйтесь модулем torchvision.transforms или библиотекой [albumentations](https://github.com/albumentations-team/albumentations)
 - (ещё раз) Можно файнтюнить предобученные модели из `torchvision`.
 - Рекомендуем написать вам сначала класс-датасет (или воспользоваться классом ImageFolder), который возвращает картинки и соответствующие им классы, а затем функции для трейна по шаблонам ниже. Однако делать это мы не заставляем. Если вам так неудобно, то можете писать код в удобном стиле. Однако учтите, что чрезмерное изменение нижеперечисленных шаблонов увеличит количество вопросов к вашему коду и повысит вероятность вызова на защиту :)
 - Валидируйте. Трекайте ошибки как можно раньше, чтобы не тратить время впустую.
 - Чтобы отладить код, пробуйте обучаться на маленькой части датасета. Когда вы поняли, что смогли всё отдебажить, переходите обучению по всему датасету
 - На каждый запуск делайте ровно одно изменение в модели/аугментации/оптимайзере, чтобы понять, что и как влияет на результат.
 - Фиксируйте random seed.
 - Начинайте с простых моделей и постепенно переходите к сложным. Обучение лёгких моделей экономит много времени.
 - Ставьте расписание на learning rate. Уменьшайте его, когда лосс на валидации перестаёт убывать.
 - Советуем использовать гпу. Если у вас его нет, используйте google colab. Если вам неудобно его использовать на постоянной основе, напишите и отладьте весь код локально на CPU, а затем запустите уже написанный ноутбук в колабе. Авторское решение задания достигает требуемой точности в колабе за 15 минут обучения.
 
Good luck & have fun! :)

In [ ]:
# Качаем данные
!wget -O dataset.zip "https://s249vla.storage.yandex.net/rdisk/6260b5c0e6e36b1584b9918ac76674f979df9b6b445741ed32ad9c35b520acdd/6060a378/_YfpybMoWZm47imviby-3WqJN6rDbfJY7vCqrcpx6a1QDycbma4FNV4faVohaeNZIqn9UfsmyNsUMG-JPSYUBw==?uid=293401772&filename=dataset.zip&disposition=attachment&hash=&limit=0&content_type=application%2Fzip&owner_uid=293401772&fsize=220318689&hid=7a4d50b29200d866f74e5c022ad9cea5&media_type=compressed&tknv=v2&etag=7531294c9be2e0a1408b50f5ff0a7b3a&rtoken=2hs2EdXLPukw&force_default=yes&ycrid=na-2c3a3d896ec0e56478abeb50a553ec82-downloader8e&ts=5be9a9655ce00&s=c69764015917cce1d6835e2c63f3cc13f8eb21867f4bdb769e1ba367807a8959&pb=U2FsdGVkX1-2RJvRHmG0jtQpETMqO97b1iqmt-zkrvHvmm4Gf_stjJk5YSp_x82obEHLr1bFchLeDXgEbYXXyReNWzpaA4tCVp4b_2H_iWo" && unzip dataset.zip

In [214]:
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
import numpy as np
import PIL
import glob
from torch.utils.data import Dataset, DataLoader
import datetime
from torch.utils.tensorboard import SummaryWriter
from torchvision.models import resnet18
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import accuracy_score
from torchvision import transforms
import random

torch.manual_seed(1)

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform):
      self.data_dir = data_dir
      self.transform = transform
      self.dirs = []
      for i_class in range(200):
        for f in glob.iglob(data_dir + "/class_" + "{0:03}".format(i_class) + "/*"):
          self.dirs.append(f) 
          # print(f)

    def __getitem__(self, idx):
      f = self.dirs[idx]
      image = self.transform(np.array(PIL.Image.open(f).convert('RGB')))
      label = int(f.split('/')[-2][-3:])
      return (image, label)


    def __len__(self):
      return len(self.dirs)


In [215]:
random.seed(1)

train_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [0.25, 0.25, 0.25], transforms.RandomVerticalFlip())])
val_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [0.25, 0.25, 0.25])])

train_dataset = MyDataset("/content/dataset/dataset/train", transform=train_transform)
val_dataset = MyDataset("/content/dataset/dataset/val", transform=val_transform)

In [216]:
# Just very simple checks
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[1][1], int)
print("tests passed")

tests passed


In [217]:
import tqdm
from sklearn.metrics import accuracy_score

def train_one_epoch(model, train_dataloader, criterion, optimizer, summary_writer, global_step, device="cuda:0"):
    
    model.train()
    progress_bar = tqdm.notebook.tqdm(train_dataloader)
    for i, (images, target) in enumerate(progress_bar):
        
        images = images.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        logits = model.forward(images)
        loss = criterion(logits, target)
        loss.backward()
        optimizer.step()
        progress_bar.set_description("Loss={:.4f}".format(loss.item()))

        global_step+=1
        if global_step % 200 == 0:
            acc = (logits.argmax(1) == target.reshape(-1)).float().mean()
            writer.add_scalar("train/loss", loss, global_step)
            writer.add_scalar("train/accuracy", acc, global_step)
        
    return global_step



def predict(model, val_dataloder, criterion, summary_writer, global_step, device="cuda:0"):
    model.eval()  
    model.to(device)
    all_losses = []
    predicted_labels = []
    true_labels = []
    accs =[]

    progress_bar = tqdm.notebook.tqdm(val_dataloder)
    for i, (images, target) in enumerate(progress_bar):
      
      images = images.to(device)
      target = target.to(device)
      logits = model.forward(images)
      loss = criterion(logits, target)
      labels = logits.argmax(dim=1).tolist()
       
      all_losses.append(loss.item())
      predicted_labels+=labels    
      true_labels+=list(map(int, target))

    return all_losses, predicted_labels, true_labels


def train(model, train_dataloader, val_dataloader, criterion, optimizer, summary_writer, device="cuda:0", n_epochs=10, scheduler=None):
    
    global_step = 0
    model.to(device)
    for epoch in range(n_epochs):
      global_step = train_one_epoch(model, train_dataloader, criterion, optimizer, summary_writer, global_step, device)
      all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, summary_writer, global_step, device)
      scheduler.step(np.mean(all_losses))
      print('Validation Accuracy = {:.4f}'.format(accuracy_score(predicted_labels, true_labels)))

In [218]:
class MyModel(nn.Module):
    def __init__(self, n_classes=10):
        super(MyModel, self).__init__()
        self.n_classes = n_classes

        self.model = nn.Sequential(
            nn.Conv2d(3, 16, (2, 2)),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Conv2d(16, 32, (2, 2)),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Conv2d(32, 64, (2, 2)),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Flatten(), # Вытянуть картинку в вектор
            nn.Linear(3136, 128),
            nn.ReLU(),
            nn.Linear(128, self.n_classes)
        )

    def forward(self, x):
        if isinstance(x, np.ndarray):
            x = torch.tensor(x)
        return self.model(x)

In [219]:
# model = MyModel(200)

# model = resnet18(pretrained=True)
# model.fc = nn.Linear(512, 200)

# model = model.to(device)
# for param in model.parameters():
#     param.requires_grad = False
# for param in model.fc.parameters():
#     param.requires_grad = True

model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = nn.Linear(1280, 200)
param_groups = [
                {'params': model._fc.parameters(), 'lr': 1e-3},
                {'params': [param for name, param in model.named_parameters() if not name.startswith('_fc')], 'lr':1e-5}
]

# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(param_groups)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=1)
n_epochs = 10
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

Loaded pretrained weights for efficientnet-b0


Простой тест на проверку правильности написанного кода

In [220]:
%load_ext tensorboard

date = datetime.datetime.now().strftime("%Y.%m.%d-%H:%M:%S")
logdir = "logs_{}".format(date)
writer = SummaryWriter(logdir)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [221]:
all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, writer, 0, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(predicted_labels, true_labels)
print("tests passed")


tests passed


Запустить обучение можно в ячейке ниже.

In [222]:
writer = SummaryWriter("{}_{}".format(datetime.datetime.now().strftime("%Y.%m.%d-%H:%M:%S"), 
                                      'EfficientNet_randomflip'))

train(model, train_dataloader, val_dataloader, criterion, optimizer, writer, device, n_epochs, scheduler)


Validation Accuracy = 0.2974



Validation Accuracy = 0.3672



Validation Accuracy = 0.3950



Validation Accuracy = 0.4223



Validation Accuracy = 0.4423



Validation Accuracy = 0.4614



Validation Accuracy = 0.4684



Validation Accuracy = 0.4804



Validation Accuracy = 0.4856



Validation Accuracy = 0.4882


После всех экспериментов которые вы проделали, выберите лучшую из своих моделей, запустите функцию evaluate. Эта функция должна брать на вход модель и даталоадер с валидационными данными и возврашать accuracy, посчитанную на этом датасете.

In [224]:
all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, writer, 0, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(true_labels, predicted_labels)
print("Оценка за это задание составит {} баллов".format(min(5, 5*accuracy / 0.44)))


Оценка за это задание составит 5 баллов


__Ваш отчёт о проделанных экспериментах__: 

Итоговый вариант:
- Предобученный EfficientNet-b0 + FC слой на наши 200 классов
- Adam с разными LR для исходной сети и последнего слоя
- Из аугментаций только нормализация + вертикальный RandomFlip
- ReduceLROnPlateau с patience = 1

Как я докатился до такого?

Почти весь код понадерган из семинаров. После того, как заработало обучение, я для начала потестил нашу рукописную модельку (см. MyModel). Она вышла слабенькой. Докинул случайную аугментацию - немного помогло.

После этого решил сразу из артилерии - подгрузил предобученные сетки: ResNet18 и EfficientNet-B0 + Adam. Вторая комбинация (уже после 5 эпохи) дала необходимый скор, на чем я и остановился.

Саммари: Упомянутых на лекциях и семинарах штук хватает для получения требуемого скора в разумное время -> эксперименты не сильно нужны. Можно было бы поиграть и получить сильно больше (0.88 упомянуты), например с помощью мощных аугментаций и игры с optimizer. Но, к сожалению, я не успел:(